### (1)Datasets简介
- 简介
    - datasets库是一个非常简单易用的数据集加载库，可以方便快捷的从本地或者HuggingFace Hub加载数据集
- 公开数据集地址
    - https://huggingface.co/datasets
- 文档地址
    - https://huggingface.co/docs/datasets/index

### (2)Datasets基本使用
- 加载在线数据集（load_dataset）
- 加载数据集某一项任务（load_dataset）
- 按照数据集划分进行加载（load_dataset）
- 查看数据集（index and slice）
- 数据集划分（train_test_split）
- 数据选取与过滤（select and filter）
- 数据映射（map）
- 保存与加载（save_to_disk/load_from_disk）

#### 加载在线数据集

In [ ]:
from datasets import *

In [ ]:
datasets = load_dataset("madao33/new-title-chinese")
datasets

#### 加载数据集合集中的某一项任务

In [ ]:
boolq_dataset=load_dataset('super_glue','boolq')
boolq_dataset

#### 按照数据集划分进行加载

In [ ]:
dataset = load_dataset("madao33/new-title-chinese", split="train")
dataset

In [ ]:
dataset = load_dataset("madao33/new-title-chinese", split="train[10:100]")
dataset

In [ ]:
dataset = load_dataset("madao33/new-title-chinese", split="train[:50%]")
dataset

In [ ]:
dataset = load_dataset("madao33/new-title-chinese", split=["train[:50%]", "train[50%:]"])
dataset

#### 查看数据集

In [ ]:
datasets = load_dataset("madao33/new-title-chinese")
datasets

In [ ]:
datasets['train']

In [ ]:
datasets['train'][0:3]

In [ ]:
datasets['train'].features

In [ ]:
datasets['train'].column_names

#### 数据集划分

In [ ]:
datasets['train'].train_test_split(test_size=0.1)

In [ ]:
dataset=boolq_dataset['train']
dataset.train_test_split(test_size=0.1,stratify_by_column='label')#分类数据集可以按照比例划分

#### 数据选取与过滤

In [ ]:
# 选取
datasets['train'].select([0,1])

In [ ]:
# 过滤(只保留含有中国的句子)
filter_dataset=datasets['train'].filter(lambda example:'中国' in example['title'])

In [ ]:
filter_dataset['title'][0:6]

#### 数据映射

In [ ]:
def add_prefix(example):
    example['title']='Prefix:' + example['title']
    return example

In [ ]:
prefix_dataset=datasets.map(add_prefix)
prefix_dataset['train']['title'][:3]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
def preprocess_function(example, tokenizer=tokenizer):
    model_inputs = tokenizer(example["content"], max_length=512, truncation=True)
    labels = tokenizer(example["title"], max_length=32, truncation=True)
    # label就是title编码的结果
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
processed_datasets=datasets.map(preprocess_function)
preprocess_function

In [ ]:
# num_proc多线程
processed_datasets = datasets.map(preprocess_function, num_proc=4)
processed_datasets

In [ ]:
# 批处理
processed_datasets = datasets.map(preprocess_function, batched=True)
processed_datasets

In [ ]:
# 去除原始字段
processed_datasets = datasets.map(preprocess_function, batched=True, remove_columns=datasets["train"].column_names)
processed_datasets

#### 保存和加载

In [ ]:
processed_datasets.save_to_disk("./processed_data")

In [ ]:
processed_datasets = load_from_disk("./processed_data")
processed_datasets

### (3)Datasets加载本地数据集

#### 直接加载文件作为数据集

In [ ]:
dataset=load_dataset('csv',data_files='./ChnSentiCorp_htl_all.csv')
dataset

In [ ]:
dataset=load_dataset('csv',data_files='./ChnSentiCorp_htl_all.csv',split='train')
dataset

In [ ]:
dataset = Dataset.from_csv("ChnSentiCorp_htl_all.csv")
dataset

#### 加载文件夹内全部文件作为数据集

In [ ]:
dataset = load_dataset("csv", data_files=["./all_data/ChnSentiCorp_htl_all.csv", "./all_data/ChnSentiCorp_htl_all copy.csv"], split='train')
dataset

#### 通过预先加载的其他格式转换加载数据集

In [ ]:
import pandas as pd

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data.head()

In [ ]:
dataset = Dataset.from_pandas(data)
dataset

In [ ]:
# List格式的数据需要内嵌{}，明确数据字段
data = [{"text": "abc"}, {"text": "def"}]
# data = ["abc", "def"]
Dataset.from_list(data)

#### 通过自定义加载脚本加载数据集

In [ ]:
load_dataset("json", data_files="./cmrc2018_trial.json", field="data")


In [ ]:
dataset = load_dataset("./load_script.py", split="train")
dataset

In [ ]:
dataset[0]

### (4)Datasets+DataCollator模型微调代码优化

In [ ]:
from transformers import  DataCollatorWithPadding

In [ ]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split='train')
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

In [ ]:
def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

In [ ]:
tokenized_dataset = dataset.map(process_function, batched=True, remove_columns=dataset.column_names)
tokenized_dataset

In [ ]:
print(tokenized_dataset[:3])

In [ ]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:

from torch.utils.data import DataLoader

In [ ]:

dl = DataLoader(tokenized_dataset, batch_size=4, collate_fn=collator, shuffle=True)

In [ ]:
num = 0
for batch in dl:
    print(batch["input_ids"].size())
    num += 1
    if num > 10:
        break